# Learning Using NTK

## Init

In [2]:
#Autocomplete
%config Completer.use_jedi = False

In [3]:
%matplotlib inline
%time

import numpy as np
import matplotlib.pyplot as plt
import os, time, glob
from tqdm.notebook import tqdm
from sklearn import svm

Wall time: 0 ns


In [4]:
os.environ['KERAS_BACKEND'] = "tensorflow"

import keras
from keras import backend as K
import tensorflow as tf

## Load Data

In [5]:
def create_cifar():
    cifar = tf.keras.datasets.cifar10
    (train_images, train_labels), (test_images, test_labels) = cifar.load_data()
    
    train_images = train_images.astype("float32") / 255.0
    test_images = test_images.astype("float32") / 255.0
    
    train_labels = np.squeeze(train_labels)
    test_labels = np.squeeze(test_labels)

    return train_images, train_labels, test_images, test_labels

In [6]:
train_x,train_y,test_x,test_y = create_cifar()
train_x.shape, test_x.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

## Run CNTK

In [ ]:
def trans(s, t, l, r, il, ir):
    """
    s,t are 4D matrices
    l,r,il,ir are 2D matrices
    """
    
    

In [ ]:
void trans(float s[32][32][32][32], float t[32][32][32][32], const float l[32][32], const float r[32][32], const float il[32][32], const float ir[32][32])
{
	int x1 = blockIdx.x;
	int y1 = blockIdx.y;
	int x2 = threadIdx.x + ((blockIdx.z >> 2) << 3);
	int y2 = threadIdx.y + ((blockIdx.z & 3) << 3);
	float S = s[x1][y1][x2][y2], T = t[x1][y1][x2][y2], L = l[x1][y1], R = r[x2][y2], iL = il[x1][y1], iR = ir[x2][y2];
	S = S * iL * iR;
	float BS = (S * (3.141592654f - acosf(max(min(S, 1.0f), -1.0f))) + sqrtf(1.0f - min(S * S, 1.0f))) * L * R / 28.274333882308138f;
	S = (3.141592654f - acosf(max(min(S, 1.0f), -1.0f))) / 28.274333882308138;
	t[x1][y1][x2][y2] = T * S + BS;
	s[x1][y1][x2][y2] = BS;

In [ ]:
def conv3(s, t):
    """
    s,t are 4D matrices
    """
    blocks=63
    threads=32
    for i in range(blocks):
        for j in range(threads):
            x1 = i + j - 31
            y1 = 

In [ ]:
void conv3(const float s[32][32][32][32], float t[32][32][32][32])
{
    int x1 = threadIdx.x + blockIdx.x - 31;
    int y1 = threadIdx.y + blockIdx.y - 31;
    int x2 = threadIdx.x;
    int y2 = threadIdx.y;

    __shared__ float d[32 + 2][32 + 2];
    if (x2 == 0){
        d[0][y2 + 1] = d[33][y2 + 1] = 0;
        if (x2 == 0 && y2 == 0)
            d[0][0] = d[0][33] = d[33][0] = d[33][33] = 0; 
    }
    if (y2 == 0){
        d[x2 + 1][0] = d[x2 + 1][33] = 0;
    }

    if (x1 < 0 || x1 > 31 || y1 < 0 || y1 > 31){
        d[x2 + 1][y2 + 1] = 0;
        return;
    }
    else
        d[x2 + 1][y2 + 1] = s[x1][y1][x2][y2];
    __syncthreads();

    t[x1][y1][x2][y2] = d[x2][y2] + d[x2][y2 + 1] + d[x2][y2 + 2]
                      + d[x2 + 1][y2] + d[x2 + 1][y2 + 1] + d[x2 + 1][y2 + 2]
                      + d[x2 + 2][y2] + d[x2 + 2][y2 + 1] + d[x2 + 2][y2 + 2];

Calculate diagonal entries of $\Sigma^{(h)}(x, x)$ and their reciprocals. See Section 4.3 in our paper. 

In [8]:
def xx(x):
    RL = [1.0, ]
    iRL = [1.0, ]

    S = np.matmul(x.T, x).reshape(32, 32, 32, 32)
    conv3(conv_blocks, conv_threads, (S, S))
    T = np.zeros((32, 32, 32, 32), dtype = np.float32)
    if not fix:
        T += S

    for i in range(1, d - 1):
        L = np.sqrt(np.diag(S.reshape(1024, 1024)).reshape(32, 32))
        iL = 1.0 / L
        RL.append(L)
        iRL.append(iL)
        trans(trans_blocks, trans_threads, (S, T, L, L, iL, iL))
        conv3(conv_blocks, conv_threads, (S, S))
        conv3(conv_blocks, conv_threads, (T, T))

    L = np.sqrt(np.diag(S.reshape(1024, 1024)).reshape(32, 32))
    iL = 1.0 / L
    RL.append(L)
    iRL.append(iL)
    trans(trans_blocks, trans_threads, (S, T, L, L, iL, iL))

    if fix:
        T -= S
    return RL, iRL

Caclulate the kernel value of x and z.

Lx and Lz are diagonal entries of $\Sigma^{(h)}(x, x)$ and $\Sigma^{(h)}(z, z)$. 

iLx and iLz are reciprocals of diagonal entries of $\Sigma^{(h)}(x, x)$ and $\Sigma^{(h)}(z, z)$. 

In [10]:
def xz(x, z, Lx, Lz, iLx, iLz):
    S = np.matmul(x.T, z).reshape(32, 32, 32, 32)
    conv3(conv_blocks, conv_threads, (S, S))
    T = np.zeros((32, 32, 32, 32), dtype = np.float32)
    if not fix:
        T += S

    for i in range(1, d - 1):
        trans(trans_blocks, trans_threads, (S, T, Lx[i], Lz[i], iLx[i], iLz[i]))
        conv3(conv_blocks, conv_threads, (S, S))
        conv3(conv_blocks, conv_threads, (T, T))

    trans(trans_blocks, trans_threads, (S, T, Lx[-1], Lz[-1], iLx[-1], iLz[-1]))

    if fix:
        T -= S
    return np.mean(T) if gap else np.trace(T.reshape(1024, 1024))

In [15]:
#Load CIFAR-10.
X_train, y_train, X_test, y_test = create_cifar()
X = np.concatenate((X_train, X_test), axis = 0)
N = X.shape[0]
N_train = X_train.shape[0]
N_test = X_test.shape[0]
X = np.asarray(X).reshape(-1, 3, 1024)

In [20]:
X[0].shape

(3, 1024)

In [ ]:
#Calculate diagonal entries.
L = []
iL = []
for i in range(N):
    Lx, iLx = xx(X[i])
    L.append(Lx)
    iL.append(iLx)

In [ ]:
#####Calculate kernel values.
H = np.zeros((N, N), dtype = np.float32)
for i in range(N):
    for j in range(N):
        H[i][j] = xz(X[i], X[j], L[i], L[j], iL[i], iL[j])

In [ ]:
#Solve kernel regression
Y_train = np.ones((N_train, 10)) * -0.1
for i in range(N_train):
    Y_train[i][y_train[i]] = 0.9
u = H[N_train:, :N_train].dot(scipy.linalg.solve(H[:N_train, :N_train], Y_train))
print "test accuracy:", 1.0 * np.sum(np.argmax(u, axis = 1) == y_test) / N_test